<div style='background-color: #87ceeb;
    border: 0.5em solid black;
    border-radius: 0.5em;
    padding: 1em;'>
    <h2>Activité</h2>
    <h1>Patience des usagers au guichet</h1>
</div>

On donne la définition d'une classe `File`.

In [ ]:
class File:
    def __init__(self):
        self.liste = []
    
    def est_vide(self):
        return self.liste == []

    def enfiler(self, elem):
        self.liste.insert(0, elem)
    
    def defiler(self):
        if self.est_vide():
            raise ValueError("la file est vide")
        return self.liste.pop()
    
    def __str__(self):
        chaine = '|'
        for k in range(len(self.liste)):
            chaine += f" {self.liste[k]} |"
        return chaine

On considère un guichet unique ouvert pendant huit heures consécutives sur une journée, soit 28800 secondes.

Les usagers qui arrivent prennent un ticket numéroté et commencent à faire la queue. On note `p` la probabilité qu'un nouvel usager arrive pendant un intervalle de temps d'une seconde.

Lorsque c'est son tour d'avoir accès au guichet, le temps que l'usager y passe est un nombre de secondes aléatoire compris entre 30 et 300. On considère que tous les nombres compris entre 30 et 300 ont la même probabilité d'apparaître.

Si le temps d'attente est trop long, un usager peut partir avant d'avoir eu accès au guichet. La patience de l'usager est un nombre de secondes aléatoire compris etre 120 et 1800. On considère que tous les nombres compris entre 120 et 1800 ont la même probabilité d'apparaître.

Le but de cette activité est de simuler une journée d'activité de ce guichet et de calculer, en fonction de `p`, quelle est la proportion des usagers qui sont repartis sans avoir été servis.

**(1)** Définir une classe `Usager` dont les instances possèdent trois attributs et une méthode :
- l'attribut `num` : le numéro du ticket attribué à l'usager à son arrivée dans la file d'attente, passé en argument de la méthode constructeur,
- l'attribut `tps_arr` : l'instant où l'usager arrive dans la file d'attente, exprimé en nombre de secondes écoulées depuis l'ouverture du guichet, passé en argument de la méthode constructeur,
- l'attribut `patience` : le nombre de secondes que l'usager accepte de passer dans la file d'attente, tiré au hasard entre 120 et 1800),
- la méthode `est_parti` : elle prend en entrée un instant `tps` (exprimé en nombre de secondes écoulées depuis l'ouverture du guichet) et elle retourne `True` si l'usager n'a pas eu la patience d'attendre jusque là, et `False` sinon.

In [ ]:
from random import random, randint

In [ ]:
class Usager:
    def __init__(self, num_ticket, tps_arrivee):
        self.num = num_ticket
        self.tps_arr = tps_arrivee
        self.patience = randint(120, 1800)
    
    def est_parti(self, tps):
        return tps > self.tps_arr + self.patience

**(2)** Définir une classe `Guichet` dont les instances possèdent trois attributs et trois méthodes :
- l'attribut `nb_tickets` : le nombre de tickets distribués depuis l'ouverture du guichet,
- l'attribut `nb_non_servis` : le nombre d'usagers partis sans avoir été servis,
- l'attribut `tps_libre` : le prochain instant où le guichet sera libre, exprimé en nombre de secondes écoulées depuis l'ouverture du guichet,
- la méthode `est_libre` : elle prend en entrée un instant `tps`, et elle permet de savoir si le guichet est libre ou non,
- la méthode `nouveau_dans_file` : elle prend en entrée une file `f` et un instant `tps`, et elle permet d'accueillir un nouvel usager dans la file d'attente,
- la méthode `servir` : elle prend en entrée un instant `tps`, et elle permet d'actualiser le prochain instant où le guichet sera libre.

In [ ]:
class Guichet:
    def __init__(self):
        self.nb_tickets = 0
        self.nb_non_servis = 0
        self.tps_libre = 0
    
    def est_libre(self, tps):
        return tps >= self.tps_libre
    
    def nouveau_dans_file(self, f, tps):
        f.enfiler(Usager(self.nb_tickets, tps))
        self.nb_tickets = self.nb_tickets + 1
    
    def servir(self, tps):
        self.tps_libre = tps + randint(30, 300)

**(3)** Compléter la définition de la fonction `proportion_non_servis` :

In [ ]:
def proportion_non_servis(p):
    """
    Simule une file d'attente au guichet sur une journée et calcule le taux d'usagers repartis sans avoir été servis.
    - Entrée : p (flottant compris entre 0 et 1, probabilité qu'un usager arrive dans la file pendant une seconde donnée)
    - Sortie : (flottant, proportion d'usagers repartis sans avoir été servis)
    """
    if not 0 < p <= 1:  # assertion : p doit être dans l'intervalle ]0;1]
        raise ValueError("l'argument doit être une probabilité non nulle")
    f = File()  # création d'une file vide
    guichet = Guichet()  # création d'une instance de la classe Guichet
    for tps in range(28800):  # on parcourt les 28800 secondes où le guichet est ouvert
        if random() < p:  # condition vraie avec une probabilité égale à p
            guichet.nouveau_dans_file(f, tps)  # un nouvel usager se présente dans la file
        if guichet.est_libre(tps) and not f.est_vide():  # si le guichet est libre et la file n'est pas vide alors...
            usager = f.defiler()  # ... un numéro de ticket est appelé au guichet...
            if usager.est_parti(tps):
                guichet.nb_non_servis = guichet.nb_non_servis + 1
            else:
                guichet.servir(tps)
    while not f.est_vide():  # les usagers qui sont toujours dans la file à la fermeture...
        f.defiler()  # ... repartent sans avoir été servis
        guichet.nb_non_servis = guichet.nb_non_servis + 1
    return guichet.nb_non_servis / guichet.nb_tickets

**(4)** Effectuer une simulation dans laquelle il y a 1% de chance qu'un usager se présente pendant chaque seconde et afficher la proportion d'usagers repartis sans avoir été servis.

In [ ]:
proportion_non_servis(0.01)

**(5)** Effectuer une simulation dans laquelle un nouvel usager se présente en moyenne toutes les deux minutes et afficher la proportion d'usagers repartis sans avoir été servis.

In [ ]:
proportion_non_servis(1/120)

**(6)** En exécutant les trois cellules suivantes, représenter graphiquement 1000 simulations : sur l'axe des abscisses `p`, sur l'axe des ordonnées la proportion d'usagers repartis sans avoir été servis.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
X = []
Y = []
for val in range(1, 201):
    p = val/4000
    for _ in range(5):
        X.append(p)
        Y.append(proportion_non_servis(p))

In [ ]:
plt.figure(figsize = (12, 8))
plt.ylim(0, 1)
plt.plot(X, Y, 'k.')
plt.show()